In [11]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from functools import partial

In [12]:
default_conv2d = partial(keras.layers.Conv2D, kernel_size=3, 
                         strides=1, padding="SAME")

In [13]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu" ,**kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.strides = strides
        
        self.activation = keras.activations.get(activation)
        self.layers = [
            default_conv2d(filters=self.filters, strides=self.strides),
            keras.layers.BatchNormalization(),
            self.activation,
            default_conv2d(filters=self.filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                default_conv2d(filters=self.filters, kernel_size=1, strides=self.strides),
                keras.layers.BatchNormalization()
            ]
        
    def call(self, inputs):
        z = inputs
        for layer in self.layers:
            z = layer(z)
        skip_z = inputs
        for layer in self.skip_layers:
            skip_z = layer(skip_z)
        return self.activation(z + skip_z)

In [14]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=64, kernel_size=7, strides=2, input_shape=[224, 224, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))

prev_filter = 64
for filter_size in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if prev_filter == filter_size else 2
    model.add(ResidualUnit(filters=filter_size, strides=strides))
    prev_filter = filter_size
    
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

In [15]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_56 (Conv2D)          (None, 109, 109, 64)      9472      
                                                                 
 batch_normalization_56 (Bat  (None, 109, 109, 64)     256       
 chNormalization)                                                
                                                                 
 activation_3 (Activation)   (None, 109, 109, 64)      0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 55, 55, 64)       0         
 2D)                                                             
                                                                 
 residual_unit_24 (ResidualU  (None, 55, 55, 64)       74368     
 nit)                                                            
                                                      